## Parallel Image thresholding using CUDA C++ and Python PIL

### We aim to threshold this image of CERN, to enhance focus on the Alps and white text

<img src="img_in.jpg" align=left width="400">

In [1]:
#include <iostream>
#include <cmath>
#include <cuda_runtime.h>
#include<vector>

### We define a CUDA kernel that brightens or underexposes pixels based on a threshold value:

In [2]:
__global__ void thresholdKernel(float* input, float* output, const int width, const int height) {
    const unsigned int col = threadIdx.x + blockIdx.x * blockDim.x;
    const unsigned int row = threadIdx.y + blockIdx.y * blockDim.y;
      if (row < height && col < width) {
        
    if(input[col + row * width] > 200)
        output[col + row * width] = input[col + row * width] * 2;
    else
        output[col + row * width] = input[col + row * width] * 0.4;
    }
}

In [3]:
const int width = 512;  
const int height = 512;

In [4]:
float* h_input = new float[width * height];
float* h_output = new float[width * height];

In [5]:
!pip install Pillow

  Obtaining dependency information for Pillow from https://files.pythonhosted.org/packages/7a/07/e896b096a77375e78e02ce222ae4fd6014928cd76c691d312060a1645dfa/Pillow-10.0.1-cp310-cp310-manylinux_2_28_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 2.2 MB/s eta 0:00:00


In [6]:
!pip install numpy

  Obtaining dependency information for numpy from https://files.pythonhosted.org/packages/9b/5a/f265a1ba3641d16b5480a217a6aed08cceef09cd173b568cd5351053472a/numpy-1.26.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 1.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 2.6 MB/s eta 0:00:00


In [7]:
!pip install matplotlib

  Obtaining dependency information for matplotlib from https://files.pythonhosted.org/packages/b5/24/aaccf324ce862bb82277e8814d2aebbb2a2c160d04e95aa2b8c9dc3137a9/matplotlib-3.8.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for contourpy>=1.0.1 from https://files.pythonhosted.org/packages/f1/6b/e4b0f8708f22dd7c321f87eadbb98708975e115ac6582eb46d1f32197ce6/contourpy-1.1.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for cycler>=0.10 from https://files.pythonhosted.org/packages/e7/05/c19819d5e3d95294a6f5947fb9b9629efb316b96de511b418c53d245aae6/cycler-0.12.1-py3-none-any.whl.metadata
  Obtaining dependency information for fonttools>=4.22.0 from https://files.pythonhosted.org/packages/ac/ed/9a33eca5e2cc35dc1fea0a968509c653db9a99a5979656ae57c6c019d66b/fonttools-4.43.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Read the image in Python using PIL and convert to grayscale

In [8]:
%%python

from PIL import Image
import numpy as np


image = Image.open('img_in.jpg') 
image = image.resize((512, 512))

image_array = np.array(image)
if len(image_array.shape) == 3 and image_array.shape[2] == 3:
    image_array = np.array(image.convert('L'))

In [9]:
void displayImgArray(float* input) {
    for (int i = 0; i < 3; i++) {
        std::cout << input[i] << " "; 
    }

    std::cout << " ... ";

    for (int i = width * height - 3; i < width * height; i++) {
        std::cout << input[i] << " "; 
    }
}

### Converting the python image into a std::vector using Cppyy:

In [10]:
%%python

import cppyy

img_list = image_array.flatten().tolist()
img_vector = cppyy.gbl.std.vector['float'](img_list)

### C++ function that recives the std::vector and passes it to the CUDA function host input:

In [11]:

void setImg(const std::vector<float>& input) {
    if (h_input != nullptr) {
        delete[] h_input;
    }

    h_input = new float[input.size()];

    for (size_t i = 0; i < input.size(); i++) {
        h_input[i] = input[i]; // No casting needed
    }
}


### C++ function to return the host output to the Python side:

In [12]:
std::vector<float> getOutput() {
    std::vector<float> res;
    for (size_t i = 0; i < width * height; i++) {
        res.push_back(h_output[i]);
    }
    return res;
}

In [13]:
%%python

cppyy.gbl.setImg(img_vector)

In [14]:
displayImgArray(h_input);

126 126 125  ... 5 16 16 

### Allocating CUDA memory and calling the kernel

In [15]:
float* d_input;
float* d_output;

cudaMalloc((void**)&d_input, width * height * sizeof(float));
cudaMalloc((void**)&d_output, width * height * sizeof(float));

cudaMemcpy(d_input, h_input, width * height * sizeof(float), cudaMemcpyHostToDevice);

dim3 dimBlock(16, 16);
dim3 dimGrid((width + dimBlock.x - 1) / dimBlock.x, (height + dimBlock.y - 1) / dimBlock.y);

thresholdKernel<<<dimGrid, dimBlock>>>(d_input, d_output, width, height);

cudaMemcpy(h_output, d_output, width * height * sizeof(float), cudaMemcpyDeviceToHost);

cudaFree(d_input);
cudaFree(d_output)

### Verifying host output

In [16]:
displayImgArray(h_output);

50.4 50.4 50  ... 2 6.4 6.4 

In [17]:
std::vector<float> blurredRes = getOutput();

### Access the CUDA filter result on the Python side:

In [18]:
%%python

k = np.array(cppyy.gbl.blurredRes, dtype = np.uint8)
k.resize(512, 512)

In [19]:
%%python

Image.fromarray(k).save("img_out.jpg")

<img src="img_out.jpg" align=left width="400">
